In [5]:
import uniprot
import pprint
seqids = "NP_000508.1  NP_001018081.3".split()

pairs = uniprot.batch_uniprot_id_mapping_pairs(
  'P_REFSEQ_AC', 'ACC', seqids)

pprint.pprint(pairs, indent=2)

Fetching 2 (P_REFSEQ_AC->ACC) mappings from http://uniprot.org...
Error in fetching metadata
[]


In [6]:
uniprot_seqids = 'A0QSU3 D9QCH6 A0QL36'.split()
uniprot_data = uniprot.batch_uniprot_metadata(
    uniprot_seqids, 'cache')
pprint.pprint(mapping, indent=2)

Fetching metadata for 3 Uniprot IDs from http://uniprot.org ...
Error in fetching metadata


NameError: name 'mapping' is not defined

In [10]:
import requests as r
from Bio import SeqIO
from io import StringIO

cID='P04637'

baseUrl="http://www.uniprot.org/uniprot/"
currentUrl=baseUrl+cID+".fasta"
response = r.post(currentUrl)
cData=''.join(response.text)

Seq=StringIO(cData)
pSeq=list(SeqIO.parse(Seq,'fasta'))

ModuleNotFoundError: No module named 'Bio'

In [12]:
import pickle

with open(file='VEC_drug_feats.pkl', mode='rb') as f:
    b = pickle.load(f)
    
print(b)

{'Drug_ID': array(['DB00672', 'DB00116', 'DB06663', ..., 'DB00610', 'DB08903',
       'DB00474'], dtype=object), 'Drug': array(['CCCNC(=O)NS(=O)(=O)C1=CC=C(C=C1)Cl',
       'C1C(NC2=C(N1)N=C(NC2=O)N)CNC3=CC=C(C=C3)C(=O)N[C@@H](CCC(=O)O)C(=O)O',
       'C1[C@H](CN2[C@@H]1C(=O)N[C@H](C(=O)N[C@@H](C(=O)N[C@H](C(=O)N[C@H](C(=O)N[C@H](C2=O)CC3=CC=CC=C3)CC4=CC=C(C=C4)OCC5=CC=CC=C5)CCCCN)CC6=CNC7=CC=CC=C76)C8=CC=CC=C8)OC(=O)NCCN',
       ..., 'C[C@@H]([C@@H](C1=CC(=CC=C1)O)O)N',
       'CN(C)CC[C@@](C1=CC=CC2=CC=CC=C21)([C@H](C3=CC=CC=C3)C4=C(N=C5C=CC(=CC5=C4)Br)OC)O',
       'CCC#CC(C)C1(C(=O)NC(=O)N(C1=O)C)CC=C'], dtype=object), 'Morgan_Features': 0       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
1       [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
2       [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, ...
4       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
6       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
                              ...                        
9

In [11]:
!ungzip uniprot_sprot_varsplic.fasta.gz

/bin/bash: ungzip: 명령어를 찾을 수 없음


In [52]:
import pandas as pd
import numpy as np

DTI_table = pd.read_csv("R_DrugBank.txt", delimiter="\t", header = None)
DTI_table.columns = ["Target_ID", "Drug_ID"]
DTI_table[DTI_table["Drug_ID"] == "DB00672"]

,Target_ID,Drug_ID
0,Q16348,DB00672
539,P23219,DB00672
4527,P46059,DB00672
4970,P08183,DB00672
5222,Q09428,DB00672
5249,Q4U2R8,DB00672
5543,P11712,DB00672
5819,P33261,DB00672


In [56]:
smiles = pd.read_csv("dti2vec_drugbankid2smiles", delimiter="\t", header = None)
smiles.columns = ["Drug_ID", "Drug"]
DTI_table_1 = pd.merge(DTI_table, smiles, on = "Drug_ID").drop_duplicates()

In [73]:
seq = pd.read_csv("origin_proteins.csv").drop("Unnamed: 0",axis = 1)
seq.columns = ["Target_ID", "Target"]
DTI_table_2 = pd.merge(DTI_table_1, seq, on = "Target_ID").drop_duplicates()

In [92]:
DTI_table_2['rel'] = 1

In [91]:
import numpy as np

neg_gene = []
neg_drug = []
count = 0
while (True) :
    gene = np.random.choice(DTI_table_2["Target_ID"])
    drug = np.random.choice(DTI_table_2["Drug_ID"])
    
    if (gene, drug) not in rel_true :
        neg_gene.append(gene)
        neg_drug.append(drug)
        count +=1
    if count >= 9864 :
        break

In [81]:
def make_set_by_values(key1, key2) :
    result = set()
    for i in range(0,len(key1)) :
        result.add((key1[i], key2[i]))
    return result
rel_true = make_set_by_values(DTI_table_2["Target_ID"], DTI_table_2["Drug_ID"])



In [113]:
neg = pd.DataFrame({"Target_ID" : neg_gene, "Drug_ID" : neg_drug})
neg_seq = pd.merge(neg, seq, on = "Target_ID")
neg_smile = pd.merge(neg_seq, smiles, on = "Drug_ID").drop_duplicates()
neg_smile["rel"] = 0

In [132]:
df = pd.concat([DTI_table_2, neg_smile])

In [135]:
import pandas as pd
import numpy as np

# Load the dataset as a pandas DataFrame

# Shuffle the DataFrame rows
df = df.sample(frac=1, random_state=42).reset_index(drop=True).dropna()
df.columns = ["Target_ID", "Drug_ID", "Drug", "Target", "Y"]
# Split the dataset into training, testing, and validation sets
train_size = 0.7
test_size = 0.2
val_size = 0.1

train_split_index = int(len(df) * train_size)
test_split_index = int(len(df) * (train_size + test_size))

train_data = df.iloc[:train_split_index]
test_data = df.iloc[train_split_index:test_split_index]
val_data = df.iloc[test_split_index:]

# Print the shapes of the resulting datasets
print('Training data shape:', train_data.shape)
print('Testing data shape:', test_data.shape)
print('Validation data shape:', val_data.shape)

train_data.to_csv('vec_train_origin.csv',index=False)
test_data.to_csv('vec_test_origin.csv',index=False)
val_data.to_csv('vec_dev_origin.csv', index=False)

Training data shape: (13434, 5)
Testing data shape: (3838, 5)
Validation data shape: (1920, 5)


In [ ]:
df